In [ ]:
# Copyright 2024 Daniel Franzen and Jan Disselhoff
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# This notebook contains our winning submission to the ARC Prize 2024 Kaggle competition,
# scoring 53.5 points on the private evaluation set.
# the ARChitects (Daniel Franzen and Jan Disselhoff)

In [1]:
# THIS IS IF WE WANT TO RUN ON MORE THAN ONE GPU
NUM_GPUS=1
import os
os.environ["WANDB_DISABLED"] = "true"

if NUM_GPUS>1:
    from common_stuff import *
    if not os.path.exists(os.path.join(tmp_dir, 'unsloth_installed')):  # unsloth offline install - https://stackoverflow.com/a/51646354
        !pip uninstall --yes torch accelerate
        !pip install --no-index --find-links=/kaggle/input/unsloth-2024-9-post4/wheelhouse unsloth
        #!pip uninstall --yes accelerate fastai torch torchaudio transformers
        #!pip install --no-index --find-links=/kaggle/input/unsloth-2024-10-7/wheelhouse unsloth  # do not use grad_acc_fix - trains very slow
        #!sed -i 's/if ((post_check - pre_check) >= 1).sum() > 1:/if False:/g' /opt/conda/lib/python3.10/site-packages/unsloth/models/llama.py
        # fix delay bug in get_statistics()
        !sed -i 's/^def get_statistics():/def get_statistics():\n if False:/g' /opt/conda/lib/python3.10/site-packages/unsloth/models/_utils.py
        # fix faulty unsloth multi-gpu detection
        !sed -i "s/raise RuntimeError('Unsloth currently does not support multi GPU setups - but we are working on it!')/pass/g" /opt/conda/lib/python3.10/site-packages/unsloth/tokenizer_utils.py /opt/conda/lib/python3.10/site-packages/unsloth/models/llama.py /opt/conda/lib/python3.10/site-packages/unsloth/models/vision.py
        os.makedirs(os.path.join(tmp_dir, 'unsloth_installed'), exist_ok=True)
        print('Unsloth installed & patched.')

    for gpu in [0, 1]: 
        signal_path = f'{model_temp_storage}_gpu{gpu}_done'
        if os.path.exists(signal_path): os.rmdir(signal_path)

    if arc_test_set.is_fake:  # cleanup? (for debugging)
        #!rm -R /kaggle/temp/finetuned_model*
        #!rm -R /kaggle/temp/inference_outputs
        #!rm -R /kaggle/temp/inference_scoring
        #!ls /kaggle/temp
        pass

In [2]:
# If using the --bg flag this will execute asynchronously 
# (you will then need to uncomment the await cell)
# %%python --bg --proc train_proc0
from common_stuff import *
start_training(gpu=0)

*** Load challanges from '/home/nikolai/Desktop/Uni/1_software/arc-prize-2024-winning-solution/local_notebooks/common_stuff.py/../data/arc-prize-2024/arc-agi_evaluation_challenges.json'...


NotADirectoryError: [Errno 20] Not a directory: '/home/nikolai/Desktop/Uni/1_software/arc-prize-2024-winning-solution/local_notebooks/common_stuff.py/../data/arc-prize-2024/arc-agi_evaluation_challenges.json'

In [8]:
# %%python --bg --proc train_proc1
# from common_stuff import *
# start_training(gpu=1)

In [9]:
# %%python --bg --proc infer_proc0
from common_stuff import *
start_inference(gpu=0)

In [10]:
# %%python --bg --proc infer_proc1
# from common_stuff import *
# start_inference(gpu=1)

In [11]:
# This is only needed if we launch the pythcon process as python --bg
# proc_exit_codes = await wait_for_subprocesses(train_proc0, train_proc1, infer_proc0, infer_proc1, print_output=True or arc_test_set.is_fake)
# print(f'*** Subprocesses exit codes: {proc_exit_codes}')
# assert all(x==0 for x in proc_exit_codes)

0. Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
1. Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
1. Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
0. Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
1. Unsloth 2024.9.post4 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.
0. Unsloth 2024.9.post4 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


1. *** Load challanges from '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'...
1. *** -> Fake test set detected, setting flag 'is_fake' to True.
1. *** Load solutions from '/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json'...
1. *** Load base model and tokenizer from '/kaggle/input/wb55l_nemomini_fulleval/transformers/default/1'...
1. 🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
1. ==((====))==  Unsloth 2024.9.post4: Fast Mistral patching. Transformers = 4.44.0.
1.    \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
1. O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
1. \        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
1.  "-____-"     Free Apache license: http://github.com/unslothai/unsloth
1. Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
1. *** Create new peft model...
1. Unsloth: Casting embed_tokens to float32
1. Unsloth: Casting 

1. Map: 100%|██████████| 800/800 [00:00<00:00, 837.28 examples/s]


0. 100%|██████████| 800/800 [00:02<00:00, 380.10it/s]


1. max_steps is given, it will override any value given in num_train_epochs
0. Map: 100%|██████████| 800/800 [00:01<00:00, 692.20 examples/s]
0. max_steps is given, it will override any value given in num_train_epochs
1. ==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
1.    \\   /|    Num examples = 800 | Num Epochs = 1
1. O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
1. \        /    Total batch size = 4 | Total steps = 20
1.  "-____-"     Number of trainable parameters = 188,760,064
0. ==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
0.    \\   /|    Num examples = 800 | Num Epochs = 1
0. O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
0. \        /    Total batch size = 4 | Total steps = 20
0.  "-____-"     Number of trainable parameters = 188,760,064


...

3.  40%|████      | 2/5 [00:43<00:42, 14.08s/it]
3.  acc:   4/  4=99.9% (2-guess),   4/  4=99.9% (any); tok: 191+ 14> 14 ALL_CORRECT:0.05440|0.05441 [25ff71a9_1.transpose.rot90.permute2683945017.ex2310.out0]
3.  40%|████      | 2/5 [00:43<00:42, 14.08s/it]
3.  acc:   4/  4=99.9% (2-guess),   4/  4=99.9% (any); tok: 191+ 14> 14 ALL_CORRECT:0.02101|0.02102 [25ff71a9_1.rot90.rot90.permute3690847152.ex3102.out0]
3.  40%|████      | 2/5 [00:44<00:42, 14.08s/it]
3.  acc:   4/  4=99.9% (2-guess),   4/  4=99.9% (any); tok: 191+ 14> 14 ALL_CORRECT:0.01939|0.01939 [25ff71a9_1.transpose.rot90.rot90.permute1045962738.ex3201.out0]
0. {'loss': 0.0507, 'grad_norm': 1.1747759580612183, 'learning_rate': 1e-05, 'epoch': 0.05}
0. {'loss': 0.0506, 'grad_norm': 0.51495760679245, 'learning_rate': 1.9e-05, 'epoch': 0.1}
0. {'train_runtime': 311.8275, 'train_samples_per_second': 0.257, 'train_steps_per_second': 0.064, 'train_loss': 0.05062842071056366, 'epoch': 0.1}
0. *** -> Training took 311.8275 seconds.
0

2. Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
2. Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


3.  80%|████████  | 4/5 [01:01<00:12, 12.63s/it]
3.  acc:   6/  6=99.9% (2-guess),   6/  6=99.9% (any); tok: 214+  4>  4 bad_content:0.38986|0.38993 [27a28665_0.transpose.rot90.permute2576908143.ex0431265.out0]
3.  80%|████████  | 4/5 [01:02<00:12, 12.63s/it]
3.  acc:   6/  6=99.9% (2-guess),   6/  6=99.9% (any); tok: 214+  4>  4 ALL_CORRECT:0.57200|0.57206 [27a28665_0.rot90.rot90.permute4209781563.ex5423610.out0]
3.  80%|████████  | 4/5 [01:03<00:12, 12.63s/it]
3.  acc:   6/  6=99.9% (2-guess),   6/  6=99.9% (any); tok: 214+  4>  4 bad_content:1.58763|1.58771 [27a28665_0.rot90.rot90.permute4209781563.ex5423610.out1]
3.  80%|████████  | 4/5 [01:03<00:12, 12.63s/it]
3.  acc:   6/  6=99.9% (2-guess),   6/  6=99.9% (any); tok: 214+  4>  4 bad_content:0.10822|0.10824 [27a28665_0.transpose.rot90.rot90.permute5194638720.ex3645012.out0]
3.  80%|████████  | 4/5 [01:03<00:12, 12.63s/it]
3.  acc:   6/  6=99.9% (2-guess),   6/  6=99.9% (any); tok: 214+  4>  4 bad_content:0.12476|0.12477 [27a28665

2. Unsloth 2024.9.post4 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


3.  80%|████████  | 4/5 [01:11<00:12, 12.63s/it]
3.  acc:   6/  7=85.7% (2-guess),   6/  7=85.7% (any); tok: 214+  4>  4 bad_content:1.33314|1.33352 [27a28665_1.rot90.rot90.permute4087162539.ex5326401.out0]
3.  80%|████████  | 4/5 [01:11<00:12, 12.63s/it]
3.  acc:   6/  7=85.7% (2-guess),   6/  7=85.7% (any); tok: 214+  4>  4 bad_content:1.61439|1.61467 [27a28665_1.rot90.rot90.permute4087162539.ex5326401.out1]
3.  80%|████████  | 4/5 [01:11<00:12, 12.63s/it]
3.  acc:   6/  7=85.7% (2-guess),   6/  7=85.7% (any); tok: 214+  4>  4 bad_content:1.17974|1.17997 [27a28665_1.transpose.rot90.rot90.permute4160723958.ex5360421.out0]
3.  80%|████████  | 4/5 [01:12<00:12, 12.63s/it]
3.  acc:   6/  7=85.7% (2-guess),   7/  7=99.9% (any); tok: 214+  4>  4 ALL_CORRECT:1.57427|1.57452 [27a28665_1.transpose.rot90.rot90.permute4160723958.ex5360421.out1]
3.  80%|████████  | 4/5 [01:12<00:12, 12.63s/it]
3.  acc:   6/  7=85.7% (2-guess),   7/  7=99.9% (any); tok: 214+  4>  4 bad_content:1.61724|1.61748 [27

In [12]:
# write submission
from common_stuff import *
with RemapCudaOOM():
    model, formatter, dataset = None, MyFormatter(), None
    decoder = Decoder(formatter, arc_test_set.split_multi_replies(), n_guesses=2, frac_score=True).from_store(infer_params['store'])
    if use_aug_score or arc_test_set.is_fake: decoder.calc_augmented_scores(model=model, store=score_temp_storage, **aug_score_params)
    submission = arc_test_set.get_submission(decoder.run_selection_algo(submission_select_algo))
    with open('submission.json', 'w') as f: json.dump(submission, f)
    if arc_test_set.is_fake:
        decoder.benchmark_selection_algos(selection_algorithms)
        with open('submission.json') as f: reload_submission = json.load(f)
        print('*** Reload score:', arc_test_set.validate_submission(reload_submission))

 acc:   1.0/  1=99.9% (2-guess),   1.0/  1=99.9% (any); tok:   ?+  ?> 14 ALL_CORRECT:0.00001|0.00001 [0520fde7_0.transpose.permute4156892370.ex120.out0]
 acc:   2.0/  2=99.9% (2-guess),   2.0/  2=99.9% (any); tok:   ?+  ?> 26 ALL_CORRECT:0.00000|0.00001 [28bf18c6_0.transpose.permute4890675321.ex012.out0]
 acc:   3.0/  3=99.9% (2-guess),   3.0/  3=99.9% (any); tok:   ?+  ?> 14 ALL_CORRECT:0.00032|0.00032 [3c9b0459_0.rot90.rot90.permute7054189263.ex0231.out0]
 acc:   4.0/  4=99.9% (2-guess),   4.0/  4=99.9% (any); tok:   ?+  ?> 14 ALL_CORRECT:0.05603|0.05605 [25ff71a9_0.transpose.rot90.rot90.rot90.permute6370481259.ex2103.out0]
 acc:   5.0/  5=99.9% (2-guess),   5.0/  5=99.9% (any); tok:   ?+  ?> 14 ALL_CORRECT:0.94956|0.94968 [2013d3e2_0.transpose.rot90.rot90.rot90.permute5630492718.ex10.out0]
 acc:   5.0/  5=99.9% (2-guess),   5.0/  5=99.9% (any); tok:   ?+  ?> 14 bad_content:0.89188|1.40555 [2013d3e2_0.transpose.rot90.rot90.rot90.permute5630492718.ex10.out1]
 acc:   5.0/  5=99.9% (2-g